In [ ]:
import warnings
warnings.filterwarnings("ignore")

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
#from IPython.display import HTML, display

# .widget-label: labels at selections widgets
# .widget-select > select: text in selections
# .widget-button: button font size
# .widget-label-basic: label size at checkboxes

"""
display(HTML('''<style>
    .widget-label { font-size: 20pt; }
    .widget-select > select { font-size: 20pt; }
    .widget-button { font-size: 20pt; }
    .widget-label-basic { font-size: 20pt; }
</style>'''))
"""

# NFDMLab Demo



```
 This file is part of NFDMLab.
 
 NFDMLab is free software; you can redistribute it and/or
 modify it under the terms of the version 2 of the GNU General
 Public License as published by the Free Software Foundation.

 NFDMLab is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
 GNU General Public License for more details.

 You should have received a copy of the GNU General Public
 License along with NFDMLab; if not, write to the Free Software
 Foundation, Inc., 59 Temple Place, Suite 330, Boston, MA
 02111-1307 USA
```

#### Contributors:
 * Sander Wahls (TU Delft) 2018
 * Marius Brehler (TU Dortmund) 2018-2019

## Imports

In [ ]:
import math
import numpy as np

%matplotlib notebook
import matplotlib.pyplot as plt

import ipywidgets as widgets

import sys
sys.path.append('../')

import FNFTpy
import Helpers.plot as hplt
from Helpers.widgets import select_constellation, select_n_blocks, select_carrier_waveform, select_power_normalization, select_path_average, select_amplification, select_noise, create_hbox_modulationHeader3, create_hbox_transmitHeader, create_hbox_linkHeader, update_alpha

from Examples.GuiEtAl2018 import GuiEtAl2018
from QualityAssessment import BitErrorRatio
from QualityAssessment import ConstellationDiagram
from QualityAssessment import ModulationEfficiency

## Feature definitions

In [ ]:
available_amplification=['Lossless','EDFA','Raman']
available_power_normalization={'2.0':{'mapping':False,'factor':2.0}, '4.0':{'mapping':True,'factor':4.0}, '6.0':{'mapping':False,'factor':6.0}, '8.0':{'mapping':True,'factor':8.0}}
available_constellations={'Reshaphed 4-QAM':{'modulation':'ReshapedQAM','level':4},'Reshaphed 16-QAM':{'modulation':'ReshapedQAM','level':16},'Reshaphed 64-QAM':{'modulation':'ReshapedQAM','level':64}}
available_n_blocks=[1,5,25]

## Widgets and functions for visualization

In [ ]:
def on_alpha_changed(change):
    update_and_plot_alpha(change['new'])

def on_constellation_changed(change):
    update_and_plot_modulator(change['new'],False,selected_power_normalization.value,selected_path_average.value)

def on_power_normalization_changed(change):
    update_and_plot_modulator(selected_constellation.value,False,change['new'],selected_path_average.value)

In [ ]:
selected_amplification = select_amplification(available_amplification, default_value='EDFA')
selected_noise = select_noise()
selected_path_average = select_path_average()

hbox_link = widgets.HBox([selected_amplification, selected_noise, selected_path_average])
ui_link = widgets.VBox([create_hbox_linkHeader(),hbox_link])

selected_amplification.observe(on_alpha_changed, names='value')

In [ ]:
selected_constellation = select_constellation(available_constellations,default_value='Reshaphed 16-QAM')
selected_power_normalization = select_power_normalization(available_power_normalization, default_value='4.0')

hbox_mod = widgets.HBox([selected_constellation, selected_power_normalization])
ui_mod = widgets.VBox([create_hbox_modulationHeader3(),hbox_mod])

selected_constellation.observe(on_constellation_changed, names='value')
selected_power_normalization.observe(on_power_normalization_changed, names='value')

In [ ]:
selected_n_blocks = select_n_blocks(available_n_blocks,default_value=25)
transmit_button = widgets.Button(description="Transmit!",layout=widgets.Layout(height='60px'))
transmit_button.style.font_weight = 'bold'

hbox_transmit = widgets.HBox([selected_n_blocks, transmit_button])
ui_transmit = widgets.VBox([create_hbox_transmitHeader(),hbox_transmit])

## Core functions

In [ ]:
example = GuiEtAl2018()
res_list = []

In [ ]:
def update_and_plot_alpha(selected_amplification):
    selected_alpha = update_alpha(selected_amplification,selected_noise,selected_path_average)
    
    plt.figure(fig_alpha.number)
    plt.cla()
    plt.title('Fiber Loss/Gain')
    plt.xlabel('z in m')
    plt.ylabel('Attenuation/Gain in dB')
    
    if selected_alpha.size == 1:  
        alpha_over_z = np.ones(example.n_steps_per_span)*selected_alpha
    else:
        alpha_over_z = selected_alpha
    
    dz = example.fiber_span_length / example.n_steps_per_span
    
    
    z_array = np.linspace(0,example.fiber_span_length,example.n_steps_per_span)
    if selected_amplification == 'Raman':
        summed_attenuation_over_z = np.cumsum(selected_alpha*dz) 
    else:
        summed_attenuation_over_z = np.arange(0,example.n_steps_per_span) * dz * alpha_over_z
        

    if selected_amplification == 'EDFA':
        plt.plot(np.append(z_array,z_array[-1]),np.append(-summed_attenuation_over_z,0))
        example.post_boost = True
    else:
        plt.plot(z_array,-summed_attenuation_over_z)
        example.post_boost = False
    
    example.alpha = selected_alpha # reconfigure will be called later anyway
    


In [ ]:
def update_modulator(choosen_constellation,dummy,power_normalization,path_average,n_blocks):
    example.Ed = available_power_normalization[power_normalization]['factor']
    example.constellation_type = available_constellations[choosen_constellation]['modulation']
    example.constellation_level  = available_constellations[choosen_constellation]['level']
    #example.roll_off_factor = available_carrier_waveforms[carrier_waveform]
    example.path_average = path_average
    example.reconfigure()
    
    tx_data = example.prepare_fiber_input(n_blocks,seed=3581)
    return tx_data
    
    
def update_and_plot_modulator(choosen_constellation,carrier_waveform,power_normalization,path_average,n_blocks=1):
    tx_data = update_modulator(choosen_constellation,carrier_waveform,power_normalization,path_average,n_blocks)
    
    plt.figure(fig_vis.number)
    plt.subplot(1,2,1)
    plt.cla()
    example.constellation.show()
    
    ax2 = plt.subplot(1,2,2)
    plt.cla()
    tx_data["nfspecs"][0].show_contspec_mag(ax2)
    return

In [ ]:
def on_transmit_clicked(b):
    example.path_average = selected_path_average.value
    example.noise = selected_noise.value
    example.reconfigure()
    tx_data, rx_data = example.run(selected_n_blocks.value,seed=3581)

    tx_power_level = np.mean(np.abs(tx_data['q'])**2)
    tx_power_level_in_dBm = 10*np.log10(tx_power_level / 0.001)
    print("Tx Power Level =", tx_power_level_in_dBm, "dBm")
    print("Link distance = ", (example.fiber_span_length*example.n_spans/1000), "km")

    dispersion_length = example.Tscale**2 / np.abs(example.beta2)
    nonlinear_length = 1.0 / np.max(np.abs(tx_data['q'])**2) / example.gamma
    print("Dispersion length =", dispersion_length/1e3, "km")
    print("Nonlinear length =", nonlinear_length/1e3, "km")
    print('')
    
    # Plots
    plt.figure(fig_link.number)
    plt.subplot(1,2,1)
    plt.cla()
    hplt.plot_q(tx_data['t'], tx_data['q'], "Fiber input")
    
    plt.subplot(1,2,2)
    plt.cla()
    hplt.plot_qout(rx_data['t'], rx_data['q'], "Fiber output")
    
    # Results to list
    res_list.clear()
    res_list.append(tx_data)
    res_list.append(rx_data)
    
    return

## Fiber parameters

In [ ]:
display(ui_link)

fig_alpha = plt.figure(figsize=(8,4))

update_and_plot_alpha('EDFA')

## Modulator parameters

In [ ]:
display(ui_mod)

fig_vis = plt.figure(figsize=(12,5.5))
plt.subplot(1,2,1)
plt.subplot(1,2,2)

update_and_plot_modulator(selected_constellation.value,False,selected_power_normalization.value,selected_path_average.value)

## Simulate transmission

In [ ]:
display(ui_transmit)

fig_link = plt.figure(figsize=(12,5.5))

transmit_button.on_click(on_transmit_clicked)

## Quality Assessment

In [ ]:
tx_data = res_list[0]
rx_data = res_list[1]

#hplt.plot_contspec(xi, contspec, r"$|r(\xi)|$", new_fig=True)
#hplt.plot_q(t, q["tx"], "Fiber input", new_fig=True)
#hplt.plot_q(t, q["rx"], "Fiber output", new_fig=True)

constellation_diagram = ConstellationDiagram(example.constellation)
constellation_diagram.plot(rx_data["symbols"], new_fig=True)

bit_error_rate = BitErrorRatio(example.constellation)
ber, nerr, nbits = bit_error_rate.compute(tx_data["symbols"], rx_data["symbols"])
print("Bit error rate =", ber)

modulation_efficiency = ModulationEfficiency()
eff = modulation_efficiency.compute(tx_data["t"], tx_data["q"], rx_data["q"], nerr, nbits)
print("Modulation efficiency =", eff[0], "bits/s/Hz")